# 🔥 YOLOv12n Thermal Video Detection

This notebook demonstrates real-time inference on thermal railway videos using a fine-tuned **YOLOv12n** model.

You will:
- Download the model weights from Google Drive  
- Download sample thermal videos from Google Drive  
- Run **real-time streaming inference** inside this notebook  


# 0) ✅ Check GPU and install required packages

In [ ]:

!nvidia-smi

!pip install torch opencv-python numpy --quiet


# 1)  🔄 Clone the GitHub repository

In [ ]:
REPO_URL = "https://github.com/Simone2408/yolov12n-thermal-video-detector.git"

!git clone "$REPO_URL"
%cd yolov12n-thermal-video-detector


# 2)       ⬇️ Download YOLOv12n thermal model weights from Google Drive


In [ ]:

import os

os.makedirs("weights", exist_ok=True)

# Direct download link for best.pt
WEIGHTS_URL = "https://drive.google.com/uc?export=download&id=1V6x8ROG5AGGCQ5PUdtZkr4UnbfXqBrZb"
WEIGHTS_PATH = "weights/best.pt"

if not os.path.exists(WEIGHTS_PATH):
    print("Downloading model weights...")
    !wget -O "$WEIGHTS_PATH" "$WEIGHTS_URL"
else:
    print("Weights already exist at:", WEIGHTS_PATH)


# 3) 🎥 Download sample thermal videos (videos.zip) from Google Drive

In [ ]:
!pip install gdown --quiet

import os
import gdown
import shutil
import glob

os.makedirs("videos", exist_ok=True)

FILE_ID = "1A5t8Qhpz4BTvHzqHiuOSccsDk9zgVhhG"
VIDEOS_ZIP_PATH = "videos.zip"

# Build the Drive URL
url = f"https://drive.google.com/uc?id={FILE_ID}"

# Download videos.zip if not present
if not os.path.exists(VIDEOS_ZIP_PATH):
    print("Downloading videos.zip from Google Drive...")
    gdown.download(url, VIDEOS_ZIP_PATH, quiet=False)
else:
    print("videos.zip already exists.")

print("Extracting videos...")
!unzip -o videos.zip -d ./videos/ >/dev/null

# 🔁 Move all .mp4 files to the root 'videos/' folder
for path in glob.glob("videos/**/*.mp4", recursive=True):
    fname = os.path.basename(path)
    dst = os.path.join("videos", fname)
    if path != dst:
        print(f"Moving {path} -> {dst}")
        shutil.move(path, dst)

# 🧹 Remove extra folders like __MACOSX and nested zip structure
extra_dirs = ["videos/__MACOSX", "videos/thermal_samples_videos"]
for d in extra_dirs:
    if os.path.exists(d):
        print(f"Removing folder: {d}")
        shutil.rmtree(d, ignore_errors=True)

print("\nDone! Available sample videos in 'videos/':")
!ls -1 videos


## 📤 Optional: upload your own thermal video

If you want to test your own thermal video, you can upload it here.  
Uploaded files will be saved into the `videos/` folder.


In [ ]:
from google.colab import files
import os

os.makedirs("videos", exist_ok=True)

print("Select one or more thermal video files to upload...")
uploaded = files.upload()

for name in uploaded.keys():
    src = name
    dst = f"videos/{name}"
    os.rename(src, dst)
    print(f"Saved: {dst}")

print("Current videos in folder:")
!ls -1 videos


In [ ]:
# 🔌 Import the real-time and offline inference functions

from src.video_inference import run_inference_colab_stream, run_inference_to_file


## 🚀 Run Real-Time Streaming Inference

Choose one of the available videos in the `videos/` folder, for example:
- `videos/termico_sample_1.mp4`
- `videos/termico_sample_2.mp4`
- or any uploaded video  

The function below will:
- Run YOLOv12n on each frame  
- Draw colored bounding boxes for each class  
- Display each annotated frame live inside this notebook  


# 4) ▶️ Real-time streaming inference on a selected video

In [ ]:


VIDEO_PATH = "videos/termico_sample_1.mp4"  # Change this if you want

run_inference_colab_stream(
    weights="weights/best.pt",
    source=VIDEO_PATH,
    img_size=640,
    conf_thres=0.25,
    device="cuda",  # use "cpu" if GPU is not available
)
